## **Notebook Content**

Project: *Customer Prioritization Under Constraints*

File: 02_feature_engineering.ipynb <br>
Author: Bryan Melvida

Purpose:
- Ingest cleaned transactional data
- Derive behavior-based and aggregate features
- Prepare feature set for downstream modeling and analysis

Input: [`customer_cleaned.parquet`](../data/cleaned/customer_cleaned.parquet) <br>

Output: [`customer_features.parquet`](../data/preprocessed/)<br>
Related Documentation: [`feature_definitions.md`](../docs/feature_engineering/feature_definitions.md)

<br>

---

<br>